In [2]:
import pandas as pd
from sqlalchemy import create_engine
import yfinance as yf

In [3]:
user = "root"
password = "fBTJ3_a_3Rg^V!K"
host = "localhost"     # or server IP
database = "stockprice_db"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

def sql_push(df,table_name):
    df.to_sql(name = table_name, con = engine, if_exists = "replace", index = False)

def download_push(ticker, tab_name):
    symbol = yf.Ticker('IOC.NS')
    iocl_df = symbol.history(start= "2007-04-01", actions = True)
    sdf = yf.download(ticker, start= "2007-04-01")
    sdf.reset_index(inplace=True)
    sdf = pd.DataFrame(sdf)
    sql_push(sdf, tab_name)

In [4]:
download_push(ticker="IOC.NS", tab_name="iocl")

C:\Users\amjit\AppData\Local\Temp\ipykernel_6744\1945624068.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
C:\Users\amjit\AppData\Local\Temp\ipykernel_6180\1945624068.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sdf = yf.download(ticker, start= "2007-04-01")
[*********************100%***********************]  1 of 1 completed


In [6]:
input("Run the query in 'ioc table prep.sql' file in MySQL, once done Press Enter to Continue ")
log_df = pd.read_sql("SELECT date, close, log_close FROM iocl", engine)
print(log_df.head())

         date  close  log_close
0  2007-04-02  14.08       2.64
1  2007-04-03  14.04       2.64
2  2007-04-04  14.14       2.65
3  2007-04-05  14.18       2.65
4  2007-04-09  14.40       2.67


In [7]:
def index_date(df):
    rdf = df.copy()
    rdf['date'] = pd.to_datetime(df['date'])
    rdf.set_index('date', inplace = True)
    return rdf

In [8]:
stock_df = index_date(log_df)
print(stock_df.head())

            close  log_close
date                        
2007-04-02  14.08       2.64
2007-04-03  14.04       2.64
2007-04-04  14.14       2.65
2007-04-05  14.18       2.65
2007-04-09  14.40       2.67
